In [13]:
## XGBoost

import numpy as np

import time

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.metrics import f1_score, roc_curve
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

from matplotlib import pyplot as plt

## To install the library:
# conda install -c conda-forge xgboost

In [26]:
# Custom functions to measure time 

def start_timer(code_block_name):
    print("Time for "+str(code_block_name)+" :")
    start_time = time.process_time()
    return start_time
    
def stop_timer(start_time):
    print("{:.2f}".format(float(time.process_time() - start_time)*1000)+str(' ms\n'))


In [15]:
data = pd.read_pickle("./data.pkl")

# split data into X and y
X = data.loc[:, data.columns != 'bot']
Y = data.loc[:, data.columns == 'bot']

In [16]:
# split data into train and test sets
seed = 10
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

# Naive Bayes implementation (for comparison)

In [17]:
# Create Gaussian NB classifier
NBClassifier = GaussianNB().fit(X_train,y_train)

y_pred_bayes = NBClassifier.predict(X_test)

# evaluate predictions
accuracy = accuracy_score(y_test, y_pred_bayes)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

f1_base_bayes = f1_score(y_test, y_pred_bayes, average='weighted')
print("F1 Score Bayes: " + str(f1_base_bayes))

Accuracy: 70.86%
F1 Score Bayes: 0.678341970273742


/home/miguel/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


# Sklearn implementation of XGBoost

In [18]:
# fit model to training data
model = XGBClassifier()
model.fit(X_train, y_train)
print(model)

/home/miguel/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)


In [19]:
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [20]:
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 88.68%


In [21]:
print("F1 Score: "+str(f1_score(y_test, predictions, average='weighted')))

F1 Score: 0.8834550293236099


# Custom implementation of XGBoost

In [22]:
def similarity_score(residuals,iteration, splitIndex):
    residuals_left_leaf = residuals[iteration, 0:splitIndex]
    residuals_right_leaf = residuals[iteration, splitIndex:]
    
    similarity_left_leaf = (np.sum(residuals_left_leaf)*np.sum(residuals_left_leaf))/np.sum(abs(residuals_left_leaf)*(np.ones(np.shape(residuals_left_leaf))-abs(residuals_left_leaf)))
    similarity_right_leaf = (np.sum(residuals_right_leaf)*np.sum(residuals_right_leaf))/np.sum(abs(residuals_right_leaf)*(np.ones(np.shape(residuals_right_leaf))-abs(residuals_right_leaf)))

   
    return similarity_left_leaf,similarity_right_leaf 

In [23]:
def output_value(residuals,iteration, splitIndex):
    
    residuals_left_leaf = residuals[iteration, 0:splitIndex]
    residuals_right_leaf = residuals[iteration, splitIndex:]
    
    output_left_leaf = np.sum(residuals_left_leaf)/np.sum(abs(residuals_left_leaf)*(np.ones(np.shape(residuals_left_leaf))-abs(residuals_left_leaf)))
    output_right_leaf = np.sum(residuals_right_leaf)*np.sum(residuals_right_leaf)/np.sum(abs(residuals_right_leaf)*(np.ones(np.shape(residuals_right_leaf))-abs(residuals_right_leaf)))

    
    return np.concatenate((np.ones(np.shape(residuals_left_leaf))*output_left_leaf, np.ones(np.shape(residuals_right_leaf))*output_right_leaf))
    

In [24]:
def sigmoid(x):
    return 1/(1 + np.exp(-x)) 

In [261]:
# Number of iterations (number of weak learners produced)
number_iterations = 6

number_samples = np.shape(X_train)[0]

weak_learners = np.zeros(shape=number_iterations, dtype=object)

# Fi: ith additive prediction
F = np.zeros(shape=(number_iterations,number_samples), dtype=float)

# hi: ith prediction of ith weak learner 
h = np.zeros(shape=(number_iterations,number_samples), dtype=float)

# Residuals (difference between y and additive prediction Fi)
residuals = np.zeros(shape=(number_iterations,number_samples), dtype=float)

#Ouput values all leafs (weights)
output_values_all_leafs = np.zeros(shape=(number_iterations), dtype=object)



for iteration in range(number_iterations):
    
    start_time = start_timer("Calculating Fi")

    if iteration == 0:
        # F0: Initial prediction is trivial: All samples same importance
        F[iteration,:] = y_train.sum()/number_samples
        
        # Fi: ith prediction is additive (it comes from adding previous predictions)
    elif iteration == 1:        
        F[iteration,:] = sigmoid(h[0,:]/(1-h[0,:]))
        
    else: # iteration 2 and so on
        F[iteration,:] = sigmoid(h[0,:]/(1-h[0,:]) + h[1:iteration+1,:].sum(axis=0))
    
    stop_timer(start_time)
    
    start_time = start_timer("Calculating Residuals")
                                   
    # Calculate the residuals:   y - Fi        
    res_aux = y_train.to_numpy().T - F[iteration,:]    
    residuals[iteration,:]= res_aux.flatten()
    
    print(np.rint(residuals[iteration,:]))
    
    stop_timer(start_time)
    
#     similarity_gain_df = pd.DataFrame()
    
#     for splitIndex in range(1,number_samples):
    
#         similarity_left_leaf,similarity_right_leaf = similarity_score(residuals,iteration, splitIndex)
        
#         similarity_gain_df = similarity_gain_df.append({'similarity_left_leaf': similarity_left_leaf,
#                                                         'similarity_right_leaf':similarity_right_leaf,
#                                                         'gain' : similarity_left_leaf + similarity_right_leaf},
#                                                        ignore_index=True)
        
#     optimum_tree_split_index = similarity_gain_df['gain'].idxmax()
    
#     h[iteration+1,:] = output_value(residuals,iteration, optimum_tree_split_index)
        
#     F[iteration+1,:] = sigmoid(h[0,:]/(1-h[0,:]) + h[1:iteration+1+1,:].sum(axis=0))



    # For each iteration, I am going just to train a decission tree model that I will suppose as optimal
    # That means that I am skipping the part of getting the gains and the tree with maximum similarity 
    
    start_time = start_timer("Training weak learner")
    
    weak_learner = DecisionTreeClassifier(max_depth=1, max_leaf_nodes=2,criterion="entropy",max_features="sqrt")
    weak_learner_model = weak_learner.fit(X_train, np.rint(residuals[iteration,:]))
    weak_learners[iteration] = weak_learner_model
    
    # Now I am going to predict X_train according to this tree classifier     
    weak_learner_pred = weak_learner_model.predict(X_train)
    
    stop_timer(start_time)
    
    start_time = start_timer("Saving leaf for each sample in dataframe")
                                   
#     # Get in which leaf the samples end    
#     sample_leaf_df = pd.DataFrame()

#     for sample in range(np.shape(X_train)[0]):
# #         start_time2 = start_timer("Getting decision path")
#         leaf = weak_learner_model.decision_path(X_train)[sample,0] #leaf where sample ends
# #         stop_timer(start_time2)
        
# #         start_time3 = start_timer("Saving leaf")
#         sample_leaf_df = sample_leaf_df.append({'residual': residuals[iteration,sample],'leaf':leaf,
#                                                'F':F[iteration,sample]},ignore_index=True)
# #         stop_timer(start_time3)

#######################################

#     # Get in which leaf the samples end   
#     leaves = weak_learner_model.decision_path(X_train)

#     leaves_dense_aux = leaves.todense()
    
#     leaves_array = np.asarray(leaves_dense[:,2]).reshape(-1)
    
        
#######################################

    leaves_array = weak_learner_model.apply(X_train)-np.ones(np.shape(leaf_array))
    
    print(leaves_array)

#######################################

    sample_leaf_df = pd.DataFrame({'residual': residuals[iteration,:],'leaf':leaves_array,'F':F[iteration,:]}) 
    
        
    stop_timer(start_time)
    
    # For each leaf, I have to calculate the output value
    # ouput_values contains the output values for the leafs in leaf_list
    
    start_time = start_timer("Get output value for all leaves")
    
    output_values = []
    
    # Get the different leafs that there are 
    leaf_list = sample_leaf_df.leaf.unique()
    
    for value in leaf_list:
        # Get samples that go to the same leaf
        sample_leaf_value_df = sample_leaf_df[sample_leaf_df.leaf == value]
        
        start_time2 = start_timer("Lambda function")
        
        # Get Fi of those samples and calculate denominator        
        denom = sample_leaf_df["F"].apply(lambda x : x*(1-x))
        
        stop_timer(start_time2)
        
        output = (sample_leaf_value_df["residual"].sum())/denom.sum()
        
        # Technically, output values are saved in order of leafs 
        output_values.append(output)
        
    # Output values are saved for each model (a model is obtained in each iteration)                               
    output_values_all_leafs[iteration] = output_values
    
    print("Output values: "+str(output_values))
    
    stop_timer(start_time)
                                   
    # Now I need to calculate hi (the prediction for the model created in this iteration) 
    # Each sample will have an ouput value depending on the leaf where it ended  
    
    start_time = start_timer("Calculating new h")
      
    # First I initialize the column
    sample_leaf_df["output_values"]=np.zeros(len(sample_leaf_df))
    
    # The value related to one leaf has already been calculated, 
    # Now, the h[iteration,sample] is updated for samples of that leaf
    # For each leaf, I write the values of the samples
    for output,index_leaf in zip(output_values,range(len(output_values))): 
        for index in range(len(sample_leaf_df)):
            # If the sample ends in the leaf, write proper output value
            if sample_leaf_df.iloc[index, 1] == leaf_list[index_leaf]:
                sample_leaf_df.iloc[index, 3] = output 
    
    h[iteration,:] = sample_leaf_df["output_values"].to_numpy()

    
    stop_timer(start_time)

Time for Calculating Fi :
1.47 ms

Time for Calculating Residuals :
[-0. -0.  1. ... -0. -0.  1.]
0.88 ms

Time for Training weak learner :
156.67 ms

Time for Saving leaf for each sample in dataframe :
[0. 1. 0. ... 1. 0. 1.]
71.19 ms

Time for Get output value for all leaves :
Time for Lambda function :
6.30 ms

Time for Lambda function :
6.99 ms

Output values: [-0.2598075440032922, 0.259807544003292]
18.53 ms

Time for Calculating new h :
4646.39 ms

Time for Calculating Fi :
0.57 ms

Time for Calculating Residuals :
[-0. -1.  1. ... -1. -0.  0.]
0.63 ms

Time for Training weak learner :
119.90 ms

Time for Saving leaf for each sample in dataframe :
[0. 0. 0. ... 0. 0. 0.]
60.06 ms

Time for Get output value for all leaves :
Time for Lambda function :
4.46 ms

Time for Lambda function :
4.37 ms

Output values: [-1.0110779318555412, 0.0040414834897691]
12.56 ms

Time for Calculating new h :
5574.99 ms

Time for Calculating Fi :
0.79 ms

Time for Calculating Residuals :
[-0. -0.  1. 

In [256]:
y_test_pred = []
for sample in range(len(X_test)): 
    
#     print(sample)
    
    test_sample = X_test.iloc[sample,:].to_numpy()
    
    output_value_sample = []
    
    # For each weak learner, get the path (the leaf)
    for model, index in zip(weak_learners,range(np.shape(weak_learners)[0])): 
        
#         leaf = model.decision_path([test_sample])
        
#         leaf_dense = leaf.todense()

#         leaf_array = np.asarray(leaf_dense).reshape(-1)[2]
        
        leaf_array = model.apply([test_sample])-np.ones(np.shape(leaf_array))
#         print(leaf_array)
        
#         print(output_values_all_leafs[0][0])
    
        # For that particular leaf, get the output value
        output_value = output_values_all_leafs[index][int(leaf_array)]
        
        output_value_sample.append(output_value)
      

    #Calculate the sigmoid
    
    print(sum(output_value_sample))
    pred = sigmoid(sum(output_value_sample))
#     print(sum(pred))
    y_test_pred.append(pred)
    

-1.5075732828353954
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-1.5075732828353954
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.06641881369690839
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.8656635520900707
-1.5075732828353954
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-1.5075732828353954
-1.5075732828353954
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-1.5075732828353954
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.7469945076620784
-0.7469945076620784
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-1.5075732828353954
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-1.5075732828353954
-1.5075732828353954
-0.1050847769167539
-1.5075732828353954

-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-1.5075732828353954
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-1.5075732828353954
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.7469945076620784
-1.5075732828353954
-1.5075732828353954
-0.7469945076620784
-0.1050847769167539
-0.7469945076620784
-1.5075732828353954
-0.1050847769167539
-0.8656635520900707
-1.5075732828353954
-1.5075732828353954
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-1.5075732828353954
-0.1050847769167539
-1.5075732828353954
-0.1050847769167539
-1.5075732828353954
-1.5075732828353954
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-0.07880887338453237
-1.5075732828353954
-0.7469945076620784
-1.5075732828353954
-0.7469945076620784
-0.7469945076620784
-1.703792051276183


-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.07880887338453237
-0.1050847769167539
-1.5075732828353954
-1.5075732828353954
-0.1050847769167539
-0.7469945076620784
-1.5075732828353954
-1.5075732828353954
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-0.7469945076620784
-1.5075732828353954
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-1.5075732828353954
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-1.5075732828353954
-0.1050847769167539
-1.5075732828353954
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-1.5075732828353954
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.7469945076620784

-0.7469945076620784
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.7469945076620784
-0.7469945076620784
-0.1050847769167539
-0.7083285444422329
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.7469945076620784
-0.7469945076620784
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-1.5075732828353954
-1.5075732828353954
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-1.5075732828353954
-1.5075732828353954
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-1.5075732828353954
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-1.5075732828353954
-0.1050847769167539
-1.5075732828353954
-1.5075732828353954
-0.8656635520900707
-1.5075732828353954
-0.1050847769167539
-1.5075732828353954
-0.1050847769167539
-0.7469945076620784


-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.8656635520900707
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.06641881369690839
-0.7469945076620784
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-0.7469945076620784
-0.1050847769167539
-1.5075732828353954
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-1.5075732828353954
-1.5075732828353954
-0.1050847769167539
-0.1050847769167539
-0.07880887338453237
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.7469945076620784
-0.7469945076620784
-0.1050847769167539
-0.07880887338453237
-0.1050847769167539
-0.07880887338453237
-0.1050847769167539
-1.5075732828353954
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-1.5075732828353954
-0.7469945076620

...

-1.5075732828353954
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-1.5075732828353954
-0.1050847769167539
-1.5075732828353954
-1.5075732828353954
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-1.5075732828353954
-1.5075732828353954
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-1.5075732828353954
-0.1050847769167539
-0.8656635520900707
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-1.5075732828353954
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.7469945076620784
-0.8656635520900707
-0.1050847769167539
-1.5075732828353954
-0.1050847769167539


-0.7469945076620784
-0.1050847769167539
-0.7469945076620784
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-1.5075732828353954
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.8656635520900707
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-1.5075732828353954
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-1.46890731961555
-0.1050847769167539
-0.1050847769167539
-1.5075732828353954
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-0.7469945076620784
-1.46890731961555
-1.5075732828353954
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.7

-0.7469945076620784
-0.1050847769167539
-1.5075732828353954
-1.5075732828353954
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-1.703792051276183
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.7469945076620784
-1.5075732828353954
-1.5075732828353954
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-1.5075732828353954
-0.30130354535754145
-0.7469945076620784
-0.7469945076620784
-0.1050847769167539
-1.5075732828353954
-0.7469945076620784
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-1.5075732828353954
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-1.5075732828353954
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-1.5075732828353954


-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-1.5075732828353954
-0.1050847769167539
-1.5075732828353954
-1.5075732828353954
-0.1050847769167539
-1.5075732828353954
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-1.5075732828353954
-0.7469945076620784
-0.1050847769167539
-1.5075732828353954
-0.7469945076620784
-1.703792051276183
-0.7469945076620784
-0.1050847769167539
-1.5075732828353954
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-1.5075732828353954
-1.5075732828353954
-0.7469945076620784
-0.1050847769167539
-0.7469945076620784
-0.8656635520900707
-1.5075732828353954
-0.7469945076620784
-0.7469945076620784
-0.1050847769167539
-1.5075732828353954
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-0.30130354535754145
-0.7469945076620784
-0.1050847769167539
0.8029102373147222
-0.1050847769167539
-0.7469945076620784
-

-1.5075732828353954
-0.1050847769167539
-0.7469945076620784
-1.5075732828353954
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-1.5075732828353954
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-1.5075732828353954
-1.703792051276183
-1.703792051276183
-0.7469945076620784
-0.7469945076620784
-1.5075732828353954
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539
-1.5075732828353954
-0.8656635520900707
-0.8656635520900707
-0.7469945076620784
-1.5075732828353954
-0.1050847769167539
-0.07880887338453237
-0.1050847769167539
-0.07880887338453237
-0.1050847769167539
-0.7469945076620784
-1.5075732828353954
-0.1050847769167539
-1.5075732828353954
-0.1050847769167539
-0.1050847769167539
-0.7469945076620784
-0.1050847769167539
-0.1050847769167539
-0.1050847769167539


In [257]:
y_test_pred = np.rint(y_test_pred)
y_test_pred

array([0., 0., 0., ..., 0., 0., 0.])

In [258]:
np.shape(y_test_pred)

(9729,)

In [259]:
accuracy_score(y_test, y_test_pred)

0.7335800185013877

In [260]:
f1_score(y_test, y_test_pred)

0.006134969325153374

In [262]:
output_values_all_leafs

array([list([-0.2598075440032922, 0.259807544003292]),
       list([-1.0110779318555412, 0.0040414834897691]),
       list([-0.06785962562773057, -0.0074223779991774205]),
       list([-0.1274906525660914, 0.11649043212488616]),
       list([-0.03321864612241864, 0.009423829922057616]),
       list([0.01848972780092968, -0.009318832882378506])], dtype=object)